In [17]:
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import *

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 19, Finished, Available)

In [18]:
# loading data to a dataframe

df = spark.read.table('holidays.holidays_silver')

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 20, Finished, Available)

In [19]:
df.schema

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 21, Finished, Available)

StructType([StructField('Country_or_region', StringType(), True), StructField('Holiday_name', StringType(), True), StructField('Norm_Holiday_name', StringType(), True), StructField('Paid_time_off', BooleanType(), True), StructField('Country_region_code', StringType(), True), StructField('Date', DateType(), True), StructField('CreatedUTC', DateType(), True), StructField('ModifiedUTC', DateType(), True)])

In [20]:
# schema for Date dimension table

DeltaTable.createIfNotExists(spark) \
    .tableName('holidays.date_gold') \
    .addColumn('Date', DateType()) \
    .addColumn('Day', IntegerType()) \
    .addColumn('Month', IntegerType()) \
    .addColumn('Year', IntegerType()) \
    .execute()


StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 22, Finished, Available)

In [21]:
# dataframe date_gold

dfdate_gold = df.dropDuplicates(['Date']).select(col('Date'), \
dayofmonth('Date').alias('Day'), \
month('Date').alias('Month'), \
year('Date').alias('Year'), \
).orderBy('Date')

display(dfdate_gold.head(5))


StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 23, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4614d651-12f5-482d-81ab-7925322dc1a3)

In [23]:
# updates to the date_gold

deltaTable = DeltaTable.forPath(spark, 'Tables/date_gold')

dfUpdates = dfdate_gold

deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.Date = updates.Date'
  ) \
   .whenMatchedUpdate(set =
    {
  
    }
  ) \
 .whenNotMatchedInsert(values =
    {     
     'Date': 'updates.Date',
     'Day': 'updates.Day',
     'Month': 'updates.Month',
     'Year': 'updates.Year'
    }
  ) \
  .execute()

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 25, Finished, Available)

In [24]:
# schema for Country dimension table

DeltaTable.createIfNotExists(spark) \
    .tableName('holidays.country_gold') \
    .addColumn('Country_or_region', StringType()) \
    .addColumn('Country_region_code', StringType()) \
    .execute()

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 26, Finished, Available)

In [54]:
# dataframe country_gold

dfcountry_gold = df.dropDuplicates(['Country_or_region', 'Country_region_code']) \
    .select(col('Country_or_region').alias('Country_or_region'), col('Country_region_code').alias('Country_region_code'))

dfcountry_gold.show(5)


StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 56, Finished, Available)

+-----------------+-------------------+
|Country_or_region|Country_region_code|
+-----------------+-------------------+
|           Sweden|                 SE|
|         Slovenia|                 SI|
|        Australia|                 AU|
|          Hungary|                 HU|
|           Poland|                 PL|
+-----------------+-------------------+
only showing top 5 rows



In [55]:
# updates to the country_gold

deltaTable = DeltaTable.forPath(spark, 'Tables/country_gold')

dfUpdates = dfcounrty_gold

deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.Country_or_region = updates.Country_or_region AND silver.Country_region_code = updates.Country_region_code'
  ) \
   .whenMatchedUpdate(set =
    {
  
    }
  ) \
 .whenNotMatchedInsert(values =
    {     
     'Country_or_region': 'updates.Country_or_region',
     'Country_region_code': 'updates.Country_region_code',
    }
  ) \
  .execute()

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 57, Finished, Available)

In [34]:
# schema for holiname_gold dimension table

DeltaTable.createIfNotExists(spark) \
    .tableName('holidays.holiname_gold') \
    .addColumn('Holiday_name', StringType()) \
    .addColumn('Paid_time_off', BooleanType()) \
    .addColumn('Holiday_ID', LongType()) \
    .execute()

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 36, Finished, Available)

In [ ]:
#add id to holiday names
holiname_gold_with_id = holiname_gold.withColumn('Holiday_ID', monotonically_increasing_id().cast(LongType()))
dfholiname_temp = spark.read.table('holidays.holiname_gold')

MAXHolidayID = dfholiname_temp.select(coalesce(max(col('Holiday_ID')),lit(0)).alias(''))

In [38]:
# dataframe holiname_gold

dfholiname_gold  = df.dropDuplicates(['Holiday_name']) \
    .select(col('Holiday_name').alias('Holiday_name'), col('Paid_time_off').alias('Paid_time_off')) \
    .withColumn('Holiday_ID', monotonically_increasing_id().cast(LongType())) #creating IDs with monotonically_increasing_id() function

dfholiname_gold.show(7)

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 40, Finished, Available)

+--------------------+-------------+----------+
|        Holiday_name|Paid_time_off|Holiday_ID|
+--------------------+-------------+----------+
|        Jour de l'an|        false|         0|
|   Sexta-feira Santa|        false|         1|
|Easter Monday [En...|         true|         2|
|New Year Holiday ...|         true|         3|
|        Húsvét Hétfő|        false|         4|
|Nemzeti ünnep elő...|        false|         5|
|Día Nacional de l...|        false|         6|
+--------------------+-------------+----------+
only showing top 7 rows



In [41]:
# updates to the holiname_gold

deltaTable = DeltaTable.forPath(spark, 'Tables/holiname_gold')

dfUpdates = dfholiname_gold

deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.Holiday_name = updates.Holiday_name AND silver.Paid_time_off = updates.Paid_time_off AND silver.Holiday_ID = updates.Holiday_ID'
  ) \
   .whenMatchedUpdate(set =
    {
  
    }
  ) \
 .whenNotMatchedInsert(values =
    {     
     'Holiday_name': 'updates.Holiday_name',
     'Paid_time_off': 'updates.Paid_time_off',
     'Holiday_ID': 'updates.Holiday_ID'
    }
  ) \
  .execute()

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 43, Finished, Available)

In [57]:
# schema for factholidays_gold fact table

DeltaTable.createIfNotExists(spark) \
    .tableName('holidays.factholidays_gold') \
    .addColumn('Date', DateType()) \
    .addColumn('Holiday_ID', LongType()) \
    .addColumn('Country_region_code', StringType()) \
    .execute()

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 59, Finished, Available)

In [58]:
# create factholidays_gold dataframe
dffactholidays_gold = df.alias('df1') \
    .join(country_gold_temp.alias('df2'), (col('df1.Country_or_region') == col('df2.Country_or_region')), 'left') \
    .join(holiname_gold_temp.alias('df3'), (col('df1.Holiday_name') == col('df3.Holiday_name')), 'left') \
    .select(
        col('df2.Country_region_code').alias('Country_region_code'),
        col('df3.Holiday_ID').alias('Holiday_ID'),
        col('df1.Date')
    ) \
    .orderBy(col('df1.Date'), col('df2.Country_region_code'))

# Show the resulting DataFrame
dffactholidays_gold.show(10)

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 60, Finished, Available)

+-------------------+----------+----------+
|Country_region_code|Holiday_ID|      Date|
+-------------------+----------+----------+
|                 AR|       237|1970-01-01|
|                 AT|       421|1970-01-01|
|                 AU|       473|1970-01-01|
|                 BE|       161|1970-01-01|
|                 BR|       434|1970-01-01|
|                 CA|       473|1970-01-01|
|                 CH|       124|1970-01-01|
|                 CO|       237|1970-01-01|
|                 CZ|       272|1970-01-01|
|                 DK|       222|1970-01-01|
+-------------------+----------+----------+
only showing top 10 rows



In [59]:
# updates to the factholidays_gold

deltaTable = DeltaTable.forPath(spark, 'Tables/factholidays_gold')

dfUpdates = dffactholidays_gold

deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.Country_region_code = updates.Country_region_code AND silver.Holiday_ID = updates.Holiday_ID AND silver.Date = updates.Date'
  ) \
   .whenMatchedUpdate(set =
    {
  
    }
  ) \
 .whenNotMatchedInsert(values =
    {     
     'Country_region_code': 'updates.Country_region_code',
     'Holiday_ID': 'updates.Holiday_ID',
     'Date': 'updates.Date'
    }
  ) \
  .execute()

StatementMeta(, add1051c-d92a-48b8-b12a-d1c0c6d92ec3, 61, Finished, Available)